In [16]:
!pip install torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch_geometric

Found existing installation: torch-geometric 2.6.1
Uninstalling torch-geometric-2.6.1:
  Successfully uninstalled torch-geometric-2.6.1
Looking in links: https://data.pyg.org/whl/torch-2.8.0+cu126.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.9 MB/s eta 0:00:00
  Using cached torch_geometric-2.6.1-py3-none-any.whl.metadata (63 kB)
Using cached torch_geometric-2.6.1-py3-none-any.whl (1.1 MB)


In [17]:
import torch
import torch.nn.functional as F
from torch.nn import Embedding, Linear

import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
import numpy as np

import urllib.request
import zipfile

from torch_geometric.data import HeteroData
from torch_geometric.nn import GATConv, to_hetero
from torch_geometric.transforms import ToUndirected, RandomLinkSplit

In [18]:
url = 'http://files.grouplens.org/datasets/movielens/ml-100k.zip'
urllib.request.urlretrieve(url, 'ml-100k.zip')
with zipfile.ZipFile('ml-100k.zip', 'r') as zip_ref:
    zip_ref.extractall()

ratings_df = pd.read_csv('ml-100k/u.data', sep='\t', names=['user_id', 'movie_id', 'rating', 'timestamp'])
user_encoder = LabelEncoder()
movie_encoder = LabelEncoder()
ratings_df['user_idx'] = user_encoder.fit_transform(ratings_df['user_id'])
ratings_df['movie_idx'] = movie_encoder.fit_transform(ratings_df['movie_id'])

data = HeteroData()
data['user'].num_nodes = len(user_encoder.classes_)
data['movie'].num_nodes = len(movie_encoder.classes_)

data['user', 'rates', 'movie'].edge_index = torch.tensor([
    ratings_df['user_idx'].values,
    ratings_df['movie_idx'].values
], dtype=torch.long)
data['user', 'rates', 'movie'].edge_label = torch.tensor(ratings_df['rating'].values, dtype=torch.float)

data = ToUndirected()(data)

print(data)

در حال دانلود دیتاست MovieLens 100K...
دیتاست با موفقیت دانلود شد.

ساختار گراف ساخته شد:
HeteroData(
  user={ num_nodes=943 },
  movie={ num_nodes=1682 },
  (user, rates, movie)={
    edge_index=[2, 100000],
    edge_label=[100000],
  },
  (movie, rev_rates, user)={
    edge_index=[2, 100000],
    edge_label=[100000],
  }
)


In [19]:
print("Splitting the data...")
original_ratings = data['user', 'rates', 'movie'].edge_label
num_ratings = data['user', 'rates', 'movie'].num_edges
data['user', 'rates', 'movie'].edge_label = torch.arange(num_ratings)
transform = RandomLinkSplit(
    num_val=0.1,
    num_test=0.1,
    is_undirected=True,
    add_negative_train_samples=False,
    edge_types=[('user', 'rates', 'movie')],
    rev_edge_types=[('movie', 'rev_rates', 'user')],
)
train_data, val_data, test_data = transform(data)
train_data['user', 'rates', 'movie'].edge_label = original_ratings[train_data['user', 'rates', 'movie'].edge_label]
val_data['user', 'rates', 'movie'].edge_label = original_ratings[val_data['user', 'rates', 'movie'].edge_label]
test_data['user', 'rates', 'movie'].edge_label = original_ratings[test_data['user', 'rates', 'movie'].edge_label]

print("Data successfully split.")
print("\nTraining Data:\n", train_data)
print("\nTest Data:\n", test_data)

Splitting the data...
Data successfully split.

Training Data:
 HeteroData(
  user={ num_nodes=943 },
  movie={ num_nodes=1682 },
  (user, rates, movie)={
    edge_index=[2, 80000],
    edge_label=[80000],
    edge_label_index=[2, 80000],
  },
  (movie, rev_rates, user)={
    edge_index=[2, 80000],
    edge_label=[80000],
  }
)

Test Data:
 HeteroData(
  user={ num_nodes=943 },
  movie={ num_nodes=1682 },
  (user, rates, movie)={
    edge_index=[2, 90000],
    edge_label=[20000],
    edge_label_index=[2, 20000],
  },
  (movie, rev_rates, user)={
    edge_index=[2, 90000],
    edge_label=[90000],
  }
)


In [20]:
class GNNEncoder(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GATConv((-1, -1), hidden_channels)
        self.conv2 = GATConv((-1, -1), out_channels)
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

class EdgeDecoder(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.lin1 = Linear(2 * hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, 1)

    def forward(self, z_dict, edge_label_index):
        user_embed = z_dict['user'][edge_label_index[0]]
        movie_embed = z_dict['movie'][edge_label_index[1]]
        concat = torch.cat([user_embed, movie_embed], dim=-1)
        x = self.lin1(concat).relu()
        x = self.lin2(x)
        return x.view(-1)

from torch_geometric.nn import HGTConv
class Model(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.user_emb = Embedding(data['user'].num_nodes, hidden_channels)
        self.movie_emb = Embedding(data['movie'].num_nodes, hidden_channels)
        self.encoder = HGTConv(-1, hidden_channels, data.metadata(), heads=2)
        self.decoder = EdgeDecoder(hidden_channels)

    def forward(self, x_dict, edge_index_dict, edge_label_index):
        z_dict = self.encoder(x_dict, edge_index_dict)
        return self.decoder(z_dict, edge_label_index)


معماری مدل با HGTConv (برای رفع خطا) با موفقیت تعریف شد.
معماری مدل با موفقیت تعریف شد.


In [21]:
from torch_geometric.loader import DataLoader
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = DataLoader(val_data, batch_size=64, shuffle=False)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

In [22]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"استفاده از دستگاه: {device}")

model = Model(hidden_channels=64).to(device)
with torch.no_grad():
  model.user_emb.weight.data.uniform_(-0.5, 0.5)
  model.movie_emb.weight.data.uniform_(-0.5, 0.5)

optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
scheduler.step()
loss_fn = torch.nn.MSELoss()

استفاده از دستگاه: cpu


/usr/local/lib/python3.12/dist-packages/torch/optim/lr_scheduler.py:192: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


In [24]:
def train():
    model.train()
    optimizer.zero_grad()
    x_dict = {
        "user": model.user_emb.weight,
        "movie": model.movie_emb.weight
    }
    pred = model(x_dict, train_data.edge_index_dict, train_data['user', 'rates', 'movie'].edge_label_index)
    target = train_data['user', 'rates', 'movie'].edge_label
    loss = loss_fn(pred, target)
    loss.backward()
    optimizer.step()
    return float(loss)

@torch.no_grad()
def test(d):
    model.eval()
    x_dict = {
        "user": model.user_emb.weight,
        "movie": model.movie_emb.weight
    }
    pred = model(x_dict, d.edge_index_dict, d['user', 'rates', 'movie'].edge_label_index)
    target = d['user', 'rates', 'movie'].edge_label
    mse = mean_squared_error(target.cpu().numpy(), pred.cpu().numpy())
    rmse = np.sqrt(mse)

    return float(rmse)

best_val_rmse = float('inf')
patience = 10
epochs_no_improve = 0

for epoch in range(1, 51):
    loss = train()
    val_rmse = test(val_data)

    if epoch % 10 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val RMSE: {val_rmse:.4f}')

    if val_rmse < best_val_rmse:
        best_val_rmse = val_rmse
        epochs_no_improve = 0
        torch.save(model.state_dict(), 'best_model.pth')
    else:
        epochs_no_improve += 1

    if epochs_no_improve == patience:
        print(f'\nValidation RMSE did not improve for {patience} epochs. Early stopping!')
        break

model.load_state_dict(torch.load('best_model.pth'))
print("\nTraining process finished. Best model loaded.")

Starting the training process with Early Stopping...
Epoch: 010, Loss: 1.1439, Val RMSE: 0.9099

Validation RMSE did not improve for 10 epochs. Early stopping!

Training process finished. Best model loaded.
